In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [7]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [8]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [9]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, activation='relu', input_shape=input_shape))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, 1, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, 3, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(25, 5, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(5, 3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 16)        4624      
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 16)        64        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 25)        10025     
__________

In [12]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 2s 31ms/step - loss: 0.6911 - acc: 0.6997 - val_loss: 0.5824 - val_acc: 0.7539
Epoch 2/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5935 - acc: 0.7446 - val_loss: 0.5675 - val_acc: 0.7544
Epoch 3/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5532 - acc: 0.7637 - val_loss: 0.5661 - val_acc: 0.7373
Epoch 4/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5453 - acc: 0.7720 - val_loss: 0.5572 - val_acc: 0.7554
Epoch 5/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5198 - acc: 0.7881 - val_loss: 0.5209 - val_acc: 0.7749
Epoch 6/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5336 - acc: 0.7651 - val_loss: 0.5391 - val_acc: 0.7529
Epoch 7/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5213 - acc: 0.7720 - val_loss: 0.5342 - val_acc: 0.7642
Epoch 8/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5252 - a

In [13]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

CaliforniaDreamin2MP 112
Predicted:  0.9919906    Actual:  True 

HoldMeTight 136
Predicted:  0.95777184    Actual:  True 

09 Suddenly I See 60
Predicted:  2.416835e-13    Actual:  False 

IllBeThereForYou 64
Predicted:  0.013527765    Actual:  False 

BloodyMerryMorning 41
Predicted:  0.00035207745    Actual:  False 

TheBollWeevil 112
Predicted:  0.001688703    Actual:  False 

BothSidesNow 41
Predicted:  0.0002628125    Actual:  False 

ChatanoogaChooChoo 105
Predicted:  0.0016107884    Actual:  False 

ImLookingThruYou 112
Predicted:  0.07079192    Actual:  False 

HoldMeTight 68
Predicted:  0.39664847    Actual:  True 

